In [1]:
!npx esbuild ../js/widget.jsx --minify --format=esm --bundle --outdir=../static

⠙⠹⠸⠼
  ../static/widget.js    1.7mb ⚠️
  ../static/widget.css  15.1kb

⚡ Done in 147ms
⠙

In [2]:
# !cd .. && npm install @radix-ui/react-icons

In [3]:
import sys

sys.path.insert(0, "/Users/joerg/git_libs/pyiron_core")
# sys.path

In [4]:
from pyiron_workflow import Workflow, as_out_dataclass_node, as_function_node
from pyironflow.wf_extensions import get_nodes, _to_jsonifyable
import pyiron_nodes as pn
import numpy as np
import json
import pyiron_workflow.wf_graph_tools as gt
import pyiron_workflow.graph.gui as gui

In [14]:
from pyiron_workflow.graph import base
from pyironflow.reactflow import ReactFlowWidget

sub_graph = base.Graph(label="subgraph")
sub_graph += pn.atomistic.structure.build.Bulk(name='Al', label="structure2")
sub_graph += pn.atomistic.structure.transform.Repeat(label="repeat2")
sub_graph += base.GraphEdge('structure2', 'repeat2', 'structure', 'structure')

# graph = base.Graph() # base.create_graph()
graph = base.Graph(label='Workflow')
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
# graph += sub_graph
graph += pn.atomistic.engine.lammps.Code(label="lammps") 
graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge("repeat", "lammps", "structure", "structure")

# base._mark_node_as_collapsed(graph, 'lammps');

structure:  NotData
structure:  NotData


In [6]:
sub_graph.nodes['structure2'].node.outputs.__getattr__('structure')

label                                            structure
type                                     builtins.NoneType
ready                                                False
value                                                 None
node     <pyiron_workflow.simple_workflow.Node object a...
dtype: object

In [7]:
opt_graph = base._optimize_graph_connections(graph)
base._nodes_to_gui(opt_graph, remove_none=False)

removing edge GraphEdge(source='var_lammps__structure', target='InitLammps', sourceHandle='x', targetHandle='structure')
removing edge GraphEdge(source='var_lammps__potential', target='InitLammps', sourceHandle='x', targetHandle='potential')
removing edge GraphEdge(source='var_lammps__calculator', target='InitLammps', sourceHandle='x', targetHandle='calculator')
removing edge GraphEdge(source='var_lammps__working_directory', target='InitLammps', sourceHandle='x', targetHandle='working_directory')
removing edge GraphEdge(source='var_lammps__calc_mode', target='Collect', sourceHandle='x', targetHandle='calc_mode')


,id,data,position,style,targetPosition,sourcePosition,type,parentId,extent,expanded
0,structure,"{'label': 'structure', 'source_labels': ['stru...","{'x': 0, 'y': 0}","{'backgroundColor': 'rgba(0, 255, 0, 0.5)', 'h...",left,right,customNode,None,None,False
1,repeat,"{'label': 'repeat', 'source_labels': ['structu...","{'x': 220, 'y': 0}","{'backgroundColor': 'rgba(0, 255, 0, 0.5)', 'h...",left,right,customNode,None,None,False
2,lammps,"{'label': 'lammps', 'source_labels': ['generic...","{'x': 440, 'y': 0}","{'backgroundColor': 'rgba(255, 165, 0, 0.3)', ...",left,right,customNode,None,None,False


In [8]:
opt_graph.edges # does the same for expanded and collapsed node

,source,target,sourceHandle,targetHandle
0,InitLammps,Shell,path,working_directory
1,Shell,ParseLogFile,log,log_file
2,Shell,ParseDumpFile,dump,dump_file
3,ParseDumpFile,Collect,dump,out_dump
4,ParseLogFile,Collect,log,out_log
5,structure,repeat,structure,structure
6,repeat,lammps,structure,structure


In [9]:
base._get_expanded_nodes(graph)

[]

In [10]:
new_graph = base._optimize_graph_connections(graph)
new_graph.edges

removing edge GraphEdge(source='var_lammps__structure', target='InitLammps', sourceHandle='x', targetHandle='structure')
removing edge GraphEdge(source='var_lammps__potential', target='InitLammps', sourceHandle='x', targetHandle='potential')
removing edge GraphEdge(source='var_lammps__calculator', target='InitLammps', sourceHandle='x', targetHandle='calculator')
removing edge GraphEdge(source='var_lammps__working_directory', target='InitLammps', sourceHandle='x', targetHandle='working_directory')
removing edge GraphEdge(source='var_lammps__calc_mode', target='Collect', sourceHandle='x', targetHandle='calc_mode')


,source,target,sourceHandle,targetHandle
0,InitLammps,Shell,path,working_directory
1,Shell,ParseLogFile,log,log_file
2,Shell,ParseDumpFile,dump,dump_file
3,ParseDumpFile,Collect,dump,out_dump
4,ParseLogFile,Collect,log,out_log
5,structure,repeat,structure,structure
6,repeat,lammps,structure,structure


In [11]:
graph.edges

,source,target,sourceHandle,targetHandle
0,var_lammps__structure,InitLammps,x,structure
1,var_lammps__potential,InitLammps,x,potential
2,var_lammps__calculator,InitLammps,x,calculator
3,var_lammps__working_directory,InitLammps,x,working_directory
4,var_lammps__calc_mode,Collect,x,calc_mode
5,InitLammps,Shell,path,working_directory
6,Shell,ParseLogFile,log,log_file
7,Shell,ParseDumpFile,dump,dump_file
8,ParseDumpFile,Collect,dump,out_dump
9,ParseLogFile,Collect,log,out_log


In [12]:
pf.wf_widgets[0].update_gui()

NameError: name 'pf' is not defined

In [ ]:
base._get_active_edges(pf.wf_widgets[0].graph)

In [ ]:
base._get_active_nodes(pf.wf_widgets[0].graph)

In [ ]:
opt_graph = base._optimize_graph_connections(pf.wf_widgets[0].graph)
base._nodes_to_gui(opt_graph, remove_none=False)

In [ ]:
base._edges_to_gui(opt_graph, remove_none=False)

In [ ]:
pf.wf_widgets[0].graph.nodes['Plot3d'].node.inputs

In [15]:
pf = gui.PyironFlow([graph])

pf.gui

In [ ]:
xx

In [ ]:
lammps = pn.atomistic.engine.lammps.InitLammps(label="lammps")
lammps.inputs

In [ ]:
from pyiron_workflow.graph import base
from pyironflow.reactflow import ReactFlowWidget

sub_graph = base.Graph(label="subgraph")
sub_graph += pn.atomistic.structure.build.Bulk(name='Al', label="structure2")
sub_graph += pn.atomistic.structure.transform.Repeat(label="repeat2")
sub_graph += base.GraphEdge('structure2', 'repeat2', 'structure', 'structure')

# graph = base.Graph() # base.create_graph()
graph = base.Graph()
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
# graph += sub_graph
graph += pn.atomistic.engine.lammps.Code(label="lammps") 
graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge("repeat", "lammps", "structure", "structure")

# base._mark_node_as_collapsed(graph, 'lammps');


print(base.get_code_from_graph(graph))

In [ ]:
graph.edges

In [ ]:
base._edges_to_gui(graph)[0]

In [ ]:
base.GuiGraph(graph)

In [ ]:
xx

In [ ]:
graph.nodes

In [ ]:
# base._nodes_to_gui(graph)
graph.edges

In [ ]:
graph = base._update_reconnected_edges(graph)
graph.edges

In [ ]:
graph.nodes

In [ ]:
base.GuiGraph(graph)

In [ ]:
xx

In [ ]:
def get_inner_edge(graph, edge):
    inner_source = f'var_{edge.target}__{edge.targetHandle}'
    df = base._edges_to_gui(graph).df
    matching_edge = df.loc[df['source'] == inner_source].iloc[0].to_dict()
    new_edge = base.GraphEdge(edge.source, matching_edge["target"], edge.sourceHandle, matching_edge["targetHandle"]) 
    return new_edge
    

get_inner_edge(graph, graph.edges[11])


In [ ]:
graph.edges[11]

In [ ]:
graph.nodes

In [ ]:
base.get_graph_from_macro(graph.nodes['lammps'].node).label

In [ ]:
base._get_active_nodes(graph)

In [ ]:
base._find_input_nodes(graph, 2)

In [ ]:
base.GuiGraph(base.get_graph_from_macro(graph.nodes['lammps'].node))

In [ ]:
graph.nodes

In [ ]:
node_to_index = {node_label: index for index, node_label in enumerate(graph.nodes.keys()) }
node_to_index

In [ ]:
base.display_gui_data(graph)

In [ ]:
base._edges_to_gui(graph)

In [ ]:
from pyiron_workflow import Workflow

wf = Workflow("LammpsMacro")

wf.structure = pn.atomistic.structure.build.Bulk(name="Al")
wf.potential = pn.atomistic.engine.lammps.Potential()
wf.calculator = pn.atomistic.engine.lammps.CalcMD()

wf.InitLammps = pn.atomistic.engine.lammps.InitLammps(
    structure=wf.structure,
    potential=wf.potential, 
    calculator=wf.calculator,
    working_directory='test',
)

wf.Shell = pn.atomistic.engine.lammps.Shell(
    # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
    working_directory=wf.InitLammps,
)

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(log_file=wf.Shell.outputs.log)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(dump_file=wf.Shell.outputs.dump)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",  # wf.calc,
)

base.get_nodes_from_wf(wf)

In [ ]:
wf._nodes

In [ ]:
graph.nodes

In [ ]:
xx

### Test wf_graph.py

In [ ]:
from pyiron_workflow.graph.base import Graph, GraphNode, GraphEdge, GuiData
import pyiron_workflow.graph.base as base
import pyiron_nodes as pn

In [ ]:
lammps = pn.atomistic.engine.lammps.Code()
macro_graph = base.get_graph_from_macro(lammps)

# w = macro_graph.display()
# w

In [ ]:
sub_graph = Graph(label="lammps")
sub_graph.add_node( pn.atomistic.structure.build.Bulk(name='Al'), label="structure2")
sub_graph.add_node( pn.atomistic.structure.transform.Repeat(), label="repeat2")
sub_graph.add_edge('structure2', 'repeat2', 'structure', 'structure')

# TODO: copy the graph to avoid changing the original graph


graph = Graph()
graph.add_node(pn.atomistic.structure.build.Bulk(name="Al"), label="structure")
graph.add_node(pn.atomistic.structure.transform.Repeat(), label="repeat")
graph.add_node(macro_graph, label="lammps")
graph.add_node(sub_graph, label="subgraph")
graph.add_edge("structure", "repeat", "structure", "structure")

# TODO: validate consistency of edges!!
if True:
    graph.add_edge("repeat", "InitLammps", "structure", "structure")
else:
    graph._mark_node_as_collapsed('lammps')
    graph._mark_node_as_collapsed('subgraph')
    graph.add_edge("repeat", "lammps", "structure", "structure")
    

w = graph.display()

w


Strategy:
- include all edges (to collapsed and expanded elements)
- add to active ones only the appropriate ones

Tasks:
- translate edge to macro node input into edge to node in macro input

In [ ]:
graph.nodes['lammps'].node.inputs

get non-connected input and output ports of all nodes inside the macro

In [ ]:
from pyiron_workflow.wf_graph_tools import get_graph_from_wf

get_graph_from_wf(lammps._wf_macro).nodes # List

In [ ]:
get_graph_from_wf(lammps._wf_macro).edges # List

In [ ]:
macro_graph.nodes # OrderedDict

In [ ]:
macro_graph.edges # OrderedDict of GraphEdge type objects! -> small but subtile differences

In [ ]:
macro_graph.display_nodes()

macro -> workflow transformation incomplete:
- non-connected input is missing
- input ports are not included
- ports should be converted into virtual nodes, which are connected to their respective nodes


In [ ]:
import inspect

def print_function_source(func):
    source_code = inspect.getsource(func)
    print(source_code)

# Example usage
print_function_source(graph.display)

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
from pyiron_nodes.atomistic.engine.lammps import Atoms, Optional, InitLammps, Shell, ParseLogFile, ParseDumpFile, Collect
from pyiron_workflow.wf_graph_tools import get_graph_from_wf

@as_function_node("generic")
def Code(
    structure: Atoms,
    calculator, # =InputCalcStatic(),  # TODO: Don't use mutable defaults
    potential: Optional[str] = None,
    working_dir: str = "test2",
):

    from pyiron_workflow import Workflow

    print("structure: ", structure)

    wf = Workflow("LammpsMacro")

    # wf.Potential = Potential(structure=structure, name=potential)

    # wf.ListPotentials = ListPotentials(structure=structure)

   #  wf.calc = Calc(calculator)

    wf.InitLammps = InitLammps(
        structure=structure,
        potential=potential, # wf.Potential,
        calculator=calculator,
        working_directory=working_dir,
    )

    wf.Shell = Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.InitLammps,
    )

    wf.ParseLogFile = ParseLogFile(log_file=wf.Shell.outputs.log)
    wf.ParseDumpFile = ParseDumpFile(dump_file=wf.Shell.outputs.dump)
    wf.Collect = Collect(
        out_dump=wf.ParseDumpFile.outputs.dump,
        out_log=wf.ParseLogFile.outputs.log,
        calc_mode="md",  # wf.calc,
    )

    return wf.Collect

@as_function_node("lammps")
def identity(x):
    return x




#inp_port_structure = Node(label="inp_port_structure", )
out = Code()._run()

wft = out._workflow

In [ ]:
lammps = pn.atomistic.engine.lammps.Code(label="lammps") 
lammps
# base.get_graph_from_macro(lammps)

In [ ]:
lammps.

In [ ]:
base.get_graph_from_wf(wft).edges

In [ ]:
# could be easily extended to go over all open ports of the subnodes
kwargs = {}
for inp in Code().inputs.data['label']:
    inp_port_label = f"inp_port_{inp}"
    kwargs[inp] = identity(inp_port_label, label=inp_port_label)

out = Code(**kwargs)._run()
base.get_graph_from_wf(out._workflow).edges

In [ ]:
wf_graph = base.get_graph_from_wf(out._workflow)
base.GuiGraph(wf_graph)

In [ ]:
out._workflow._nodes

In [ ]:
out._workflow._edges

In [ ]:
# transfer macro to subnode ports

In [ ]:
get_graph_from_wf(wft).nodes  # straightforward approach to get the missing edges between input port and subnode inputs!
# we only lack the sub node inputs that have no default parameters, this is ok (would be easy to get them)

In [ ]:
wft.add_node(label="inp_structure", node=inp_structure)


# wft.add_edge("inp_structure", "InitLammps", "structure", "structure")

In [ ]:
wft.child_labels

In [ ]:
wft._nodes

In [ ]:
out._workflow._edges

In [ ]:
graph.nodes['lammps'].graph.nodes["InitLammps"].node.inputs

In [ ]:
for key, node in graph.nodes['lammps'].graph.nodes.items():
    print(key, node.node.inputs)

In [ ]:
graph.nodes['lammps'].graph.nodes.keys()

In [ ]:

graph.display_nodes()

In [ ]:
graph.display_active_nodes()

In [ ]:
graph.display_gui_data()

In [ ]:
graph.display_edges()

In [ ]:
graph.display_active_edges()

In [ ]:
graph._edges_to_gui()